<a href="https://colab.research.google.com/github/profteachkids/CHE3022/blob/main/TwoTanksSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.integrate import solve_ivp
import jax.numpy as jnp
import jax
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_dark'

In [3]:
tend=2000
sq2g = jnp.sqrt(2*9.81)
Ac1=1.0
Ac2=1.0
A12 = 0.03
A2 = 0.03
h1i = 5.0
h2i = 3.0

Q12i = A12 * jnp.sign(h1i-h2i) * sq2g * jnp.sqrt(jnp.abs(h1i-h2i))
Q2i = A2 * sq2g * jnp.sqrt(h2i)
Q1ini = Q12i
Q2ini = Q2i - Q12i
print(f'Q1ini={Q1ini}, Q2ini={Q2ini}')

Q1ini=0.1879255175590515, Q2ini=0.042235299944877625


In [10]:
def switch(t, v1, v2, tswitch, scale):
    return jax.nn.sigmoid((t-tswitch)/scale)*(v2-v1)+v1

In [15]:
def Q1in(t):
    return switch(t, Q1ini, 0.8*Q1ini, 500, 50)

def Q2in(t):
    return switch(t, Q2ini, 1.2*Q2ini, 700, 50)

In [26]:
@jax.jit
def dhdt(t, h):
    h1=h[0]
    h2=h[1]
    Q12 = A12 * jnp.sign(h1-h2) * sq2g * jnp.sqrt(jnp.abs(h1-h2))
    Q2 = A2 * sq2g * jnp.sqrt(h2)

    return jnp.array([(Q1in(t) - Q12)/Ac1,
                      (Q2in(t) + Q12 - Q2)/Ac2])

In [27]:

res=solve_ivp(dhdt, (0,tend), (h1i, h2i), method='Radau', dense_output=True, jac=jax.jacobian(dhdt, 1))
tplot = jnp.linspace(0,tend,501)
sol = res.sol(tplot)

In [28]:
fig = make_subplots(rows=1,cols=2)
fig.add_trace(go.Scatter(x=tplot, y=sol[0], name='h1'), row=1, col=1)
fig.add_trace(go.Scatter(x=tplot, y=sol[1], name='h2'), row=1, col=1)
fig.add_trace(go.Scatter(x=tplot, y=Q1in(tplot), name='Q1in'), row=1, col=2)
fig.add_trace(go.Scatter(x=tplot, y=Q2in(tplot), name='Q2in'), row=1, col=2)